In [1]:
from model import Model
from pydantic import BaseModel

gpt = Model('openai', 'gpt-4o-2024-08-06')
client = gpt.load_model()


class Parts(BaseModel):
    number: str
    name: str

class Models(BaseModel):
    parts: list[Parts]
    name: str

Initializing OpenAI client with API key: sk-pr...


In [2]:
case_accurate = """ 
TechCorp's latest smartphone, the UltraPhone X, was ready for production. The spec sheet listed its key components: "CPU-A15" Octa-core processor,
"RAM-8GB" high-speed memory, "CAM-48MP" main camera, and "BAT-5000" long-life battery. Quality control had approved the "SCR-6.5" OLED screen and
"SEN-FP" fingerprint sensor. The marketing team was eager to highlight these features in their upcoming campaign."""
case_hallucination = """ 
The annual tech conference was in full swing. Attendees milled about, discussing the latest trends in artificial intelligence and blockchain. 
At the coffee station, two developers debated the merits of different programming paradigms. Meanwhile, in the main hall, a keynote speaker 
was explaining the potential impact of quantum computing on cybersecurity. The air was buzzing with excitement about the future of technology."""
case_ambigious = """
At the Quantum Dynamics lab, researchers were fine-tuning their experimental quantum computer. Dr. Lee examined the system's architecture, 
which included multiple components labeled with alphanumeric codes. The Q-100 and Q-200 were crucial, but their exact roles weren't clear. 
She noted that X-1 through X-5 were connected to Y-10, forming a complex network. Z-001 seemed important, yet its relationship to QC-A and 
QC-B was uncertain. As alarms beeped, Dr. Lee wondered if the issue was with E-7 or perhaps the mysterious P-Series modules.
"""


In [3]:
def respond(user_prompt):
    response_accurate = gpt.structured_query(system_message="You are an expert parser.", user_prompt=user_prompt, response_format=Models)
    return response_accurate

def print_case(response):
    for i, parts in enumerate(response.parts):
        print("Part", i, parts)
    print("Model", response.name)

def respond_and_print(user_prompt):
    print_case(respond(user_prompt))

In [4]:
user_prompt_accurate = f"Be careful. Extract from the following data: {case_accurate}"
respond_and_print(user_prompt_accurate)

Querying gpt-4o-2024-08-06... | parts=[Parts(number='CPU-A15', name='Octa-core processor'), Parts(number='RAM-8GB', name='high-speed memory'), Parts(number='CAM-48MP', name='main camera'), Parts(number='BAT-5000', name='long-life battery'), Parts(number='SCR-6.5', name='OLED screen'), Parts(number='SEN-FP', name='fingerprint sensor')] name='UltraPhone X'
Part 0 number='CPU-A15' name='Octa-core processor'
Part 1 number='RAM-8GB' name='high-speed memory'
Part 2 number='CAM-48MP' name='main camera'
Part 3 number='BAT-5000' name='long-life battery'
Part 4 number='SCR-6.5' name='OLED screen'
Part 5 number='SEN-FP' name='fingerprint sensor'
Model UltraPhone X


In [5]:
user_prompt_hallucination = f"Be careful. Extract from the following data: {case_hallucination}"
respond_and_print(user_prompt_hallucination)

Querying gpt-4o-2024-08-06... | parts=[Parts(number='1', name='Artificial Intelligence'), Parts(number='2', name='Blockchain'), Parts(number='3', name='Programming Paradigms'), Parts(number='4', name='Quantum Computing'), Parts(number='5', name='Cybersecurity')] name='Tech Conference Topics'
Part 0 number='1' name='Artificial Intelligence'
Part 1 number='2' name='Blockchain'
Part 2 number='3' name='Programming Paradigms'
Part 3 number='4' name='Quantum Computing'
Part 4 number='5' name='Cybersecurity'
Model Tech Conference Topics


In [6]:
user_prompt_ambigious = f"Be careful. Extract from the following data: {case_ambigious}"
respond_and_print(user_prompt_ambigious)

Querying gpt-4o-2024-08-06... \ parts=[Parts(number='Q-100', name='Quantum Computer Component'), Parts(number='Q-200', name='Quantum Computer Component'), Parts(number='X-1', name='Network Component'), Parts(number='X-2', name='Network Component'), Parts(number='X-3', name='Network Component'), Parts(number='X-4', name='Network Component'), Parts(number='X-5', name='Network Component'), Parts(number='Y-10', name='Network Component'), Parts(number='Z-001', name='Quantum Computer Component'), Parts(number='QC-A', name='Quantum Computer Component'), Parts(number='QC-B', name='Quantum Computer Component'), Parts(number='E-7', name='Quantum Computer Component'), Parts(number='P-Series', name='Quantum Computer Component')] name='Quantum Dynamics Lab Components'
Part 0 number='Q-100' name='Quantum Computer Component'
Part 1 number='Q-200' name='Quantum Computer Component'
Part 2 number='X-1' name='Network Component'
Part 3 number='X-2' name='Network Component'
Part 4 number='X-3' name='Networ

In [8]:
user_prompt_hallucination_with_prompting = f"Strictly extract if the data matches the expected schema, put None if the information is not there. Extract from the following data: {case_hallucination}"
respond_and_print(user_prompt_hallucination_with_prompting)

Querying gpt-4o-2024-08-06... / parts=[Parts(number='1', name='Annual Tech Conference'), Parts(number='2', name='Artificial Intelligence'), Parts(number='3', name='Blockchain'), Parts(number='4', name='Programming Paradigms'), Parts(number='5', name='Quantum Computing'), Parts(number='6', name='Cybersecurity')] name='Tech Conference Highlights'
Part 0 number='1' name='Annual Tech Conference'
Part 1 number='2' name='Artificial Intelligence'
Part 2 number='3' name='Blockchain'
Part 3 number='4' name='Programming Paradigms'
Part 4 number='5' name='Quantum Computing'
Part 5 number='6' name='Cybersecurity'
Model Tech Conference Highlights


In [10]:
user_prompt_ambigious_with_prompting = f"Strictly extract if the data matches the expected schema, put None if the information is not there. Extract from the following data: {case_ambigious}"
respond_and_print(user_prompt_ambigious_with_prompting)

Querying gpt-4o-2024-08-06... - parts=[Parts(number='Q-100', name=''), Parts(number='Q-200', name=''), Parts(number='X-1', name=''), Parts(number='X-2', name=''), Parts(number='X-3', name=''), Parts(number='X-4', name=''), Parts(number='X-5', name=''), Parts(number='Y-10', name=''), Parts(number='Z-001', name=''), Parts(number='QC-A', name=''), Parts(number='QC-B', name=''), Parts(number='E-7', name='')] name='Quantum Dynamics lab'
Part 0 number='Q-100' name=''
Part 1 number='Q-200' name=''
Part 2 number='X-1' name=''
Part 3 number='X-2' name=''
Part 4 number='X-3' name=''
Part 5 number='X-4' name=''
Part 6 number='X-5' name=''
Part 7 number='Y-10' name=''
Part 8 number='Z-001' name=''
Part 9 number='QC-A' name=''
Part 10 number='QC-B' name=''
Part 11 number='E-7' name=''
Model Quantum Dynamics lab
